## Training a Custom TensorFlow Model in SageMaker

Let's see how to train a custom model in the cloud using SageMaker's training service.

### Resources

- [Available Docker images](https://github.com/aws/deep-learning-containers/blob/master/available_images.md)
- [Documentation of TensorFlow SageMaker Python SDK](https://sagemaker.readthedocs.io/en/stable/frameworks/tensorflow/using_tf.html#create-an-estimator)

## Setup SageMaker Permissions

In [ ]:
import sagemaker
from sagemaker import get_execution_role
role = get_execution_role()
sagemaker_session = sagemaker.Session()
region = sagemaker_session.boto_session.region_name
from sagemaker.tensorflow import TensorFlow

## Configuration

In [ ]:
from datetime import datetime
date = datetime.now().strftime("%y%m%d-%H%M%S")

image_uri_cpu='763104351884.dkr.ecr.{}.amazonaws.com/tensorflow-training:2.3.1-cpu-py37-ubuntu18.04'.format(region)
image_uri_gpu='763104351884.dkr.ecr.{}.amazonaws.com/tensorflow-training:2.3.1-gpu-py37-cu102-ubuntu18.04'.format(region)
image_uri = image_uri_gpu

epochs = 100

instance_type = 'ml.p2.xlarge' # ml.p3.2xlarge, ml.p3.16xlarge

device='gpu'

job_name = '{}-Fashion-Mnist-{}-{}-{}e'.format(
    date,
    instance_type.replace('.','-').replace('ml-', ''),
    device,
    epochs)

## Build a TensorFlow Estimator

In [ ]:
estimator = TensorFlow(entry_point='fashion_mnist.py',
                      role=role,
                      instance_count=1,
                      instance_type=instance_type,
                      image_uri=image_uri,
                      model_dir='s3://neuron-work/models-sagemaker/',
                      hyperparameters={
                          'epochs': epochs
                      },
                      script_mode=False)

In [ ]:
estimator.hyperparameters()

## Start Training Job

In [ ]:
estimator.fit('s3://neuron-work/datasets/Fashion-Mnist/',
              wait=False, # True makes notebook wait and logs output in real time
              job_name=job_name)

In [ ]:
'Done!'

In [ ]:
#p3_estimator = TensorFlow.attach('201126-191855-Fashion-Mnist-p3-16xlarge-gpu')